***Syntax natürlicher Sprachen, WS 2021/22***

---
# Übung 5

In [ ]:
import nltk
import sys
from spacy import displacy

In [ ]:
from nltk import DependencyGraph
from nltk.tree import Tree
from itertools import chain


### tree_labeled-Methode für DependencyGraphs (pretty_print-Dependenzbäume mit Labels):
def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

    
DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled



def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])





def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex

---
## Aufgabe 1:  Adjunkt-Tests


### Aufgabe 1 a: geschehens-Test

#### Gegeben sei folgender Satz, dessen drittes Satzglied den geschehens-Test besteht (Satz bleibt wohlgeformt). 

#### *den ganzen Tag* wird somit vom Test korrekt als Angabe (Adjunkt) bestätigt.


In [4]:
sentence = ["er", "wartet", "den ganzen Tag"] #wohlgeformter Ausgangssatz

sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[2]
#wohlgeformt nach Test-Anwendung:

'er wartet, und das geschieht den ganzen Tag'

--- 
#### Welche syntaktische Funktion hat das Adjunkt *den ganzen Tag*?  (Objekt oder Adverbial?) Testen Sie über Passivierung!

den ganzen Tag is adverbial

*der ganze Tag wird von ihm gewartet

---
#### Geben Sie (unter Erhalt der Wohlgeformtheit des Ausgangssatzes) ein alternatives drittes Satzglied an, so dass der geschehens-Test fehlschlägt. 


In [8]:
sentence = ["er", "wartet", "auf ihn"] #wohlgeformter Ausgangssatz

sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[2]
#nicht wohlgeformt nach Test-Anwendung:

'er wartet, und das geschieht auf ihn'

#### Für welchen Typ von Dependent spricht dies (Angabe/Adjunkt oder Ergänzung/Komplement)?


komplement

#### Ist dies auch korrekt?

- yes, PP-komplement = prepositional objet

- ich warte (auf etwas)

---
### Aufgabe 1 b: Adverbialsatz-Test

#### Funktioniert die Feststellung von *den ganzen Tag*  als Adjunkt auch mit dem Adverbialsatz-Test?


In [9]:
sentence = ["er", "wartet", "den ganzen Tag"] #wohlgeformter Ausgangssatz

sentence[0] + " " + sentence[1] + ", als er " + sentence[2] + " war"
#wohlgeformt nach Test-Anwendung??

'er wartet, als er den ganzen Tag war'

---
#### Geben Sie ein drittes Satzglied an, das den Adverbialsatz-Test besteht (Satz bleibt wohlgeformt).

#### Was ist das Ergebnis des Tests?

In [11]:
sentence = ["er", "wartet", "in dem Park"] #wohlgeformter Ausgangssatz

sentence[0] + " " + sentence[1] + ", als er " + sentence[2] + " war"
#wohlgeformt nach Test-Anwendung:

'er wartet, als er in dem Park war'

---
### Aufgabe 1 c: Eliminierungstest

#### Funktioniert die Feststellung von *den ganzen Tag* als Adjunkt auch mit dem Eliminierungstest?


In [12]:
sentence = ["er", "wartet", "den ganzen Tag"] #wohlgeformter Ausgangssatz

sentence[0] + " " + sentence[1] 
#wohlgeformt nach Test-Anwendung??

'er wartet'

---
#### Geben Sie in folgendem Beispiel ein Verb an, so dass das dritte Satzglied den Eliminierungstest NICHT besteht.

#### Was ist das Ergebnis des Tests?

In [14]:
sentence = ["er", "bekommt", "eine Nachricht"] #wohlgeformter Ausgangssatz

sentence[0] + " " + sentence[1]
#nicht wohlgeformt nach Test-Anwendung:

'er bekommt'

---
## Aufgabe 2: Dependenzgrammatik mit dem NLTK

#### Das NLTK lässt beim Schreiben von Dependenzgrammatiken lediglich die Beschreibung der Abhängigkeiten zwischen Wörtern zu. Dadurch werden Dependenzgrammatiken leicht redundant. Betrachten Sie das folgende Beispiel, das Ihnen vom letzten Übungsblatt bekannt vorkommen dürfte:

In [15]:
from nltk.parse import ProjectiveDependencyParser

grammar = nltk.DependencyGrammar.fromstring("""
'gibt' -> 'Mann' | 'Frau' | 'Buch'
'schenkt' -> 'Mann' | 'Frau' | 'Buch'
'Mann' -> 'der'
'Frau' -> 'der' | 'die'
'Buch' -> 'das'
""")

parser = ProjectiveDependencyParser(grammar)
sent = "der Mann gibt der Frau das Buch".split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

     gibt     
 ┌────┼────┐   
Mann Frau Buch
 │    │    │   
der  der  das 



---
#### Erweitern Sie die Grammatik um die Verben aus Aufgabe 2 von Übungsblatt 4:
- *schläft*
- *gefällt*
- *kennt*

#### Verwenden Sie dazu die folgende Analyse-Funktion und sammeln Sie die Sätze, die als grammatisch erkannt werden sollten, in einer Liste.

In [16]:
def test_dep_grammar(grammar, sentences):
    depgr = nltk.DependencyGrammar.fromstring(grammar)
    parser = nltk.ProjectiveDependencyParser(depgr)
    
    for i, sent in enumerate(sentences, 1):
        print("Satz {}: {}".format(i, sent))
        results = parser.parse(sent.split())
        analyzed = False
        for tree in results:
            tree.pretty_print(unicodelines=True)
            analyzed = True
        if not analyzed:
            print("Keine Analyse möglich", file=sys.stderr)

In [31]:
grammar1 = """
'gibt' -> 'Mann' | 'Frau' | 'Buch'
'schenkt' -> 'Mann' | 'Frau' | 'Buch'
'schläft' -> 'Mann' | 'Buch'
'gefällt' -> 'Frau' | 'Buch' 
'kennt' -> 'Frau' | 'Buch' 
'Mann' -> 'der'
'Frau' -> 'der' | 'die'
'Buch' -> 'das'
"""

In [28]:
test_sentences = [
    "der Mann gibt der Frau das Buch",
    "der Mann schläft",
    "das Buch gefällt der Frau",
    "die Frau kennt das Buch"    
]

In [32]:
other_sents = [
    "der Mann schläft das Buch"
]
test_dep_grammar(grammar1,other_sents)

Satz 1: der Mann schläft das Buch
     schläft     
 ┌──────┴─────┐   
Mann         Buch
 │            │   
der          das 



In [29]:
test_dep_grammar(grammar1,test_sentences)

Satz 1: der Mann gibt der Frau das Buch
     gibt     
 ┌────┼────┐   
Mann Frau Buch
 │    │    │   
der  der  das 

Satz 2: der Mann schläft
schläft
   │    
  Mann 
   │    
  der  

Satz 3: das Buch gefällt der Frau
     gefällt     
 ┌──────┴─────┐   
Buch         Frau
 │            │   
das          der 

Satz 4: die Frau kennt das Buch
     kennt     
 ┌─────┴────┐   
Frau       Buch
 │          │   
die        das 



---
#### Erweitern Sie die Grammatik nun um die Adjektive aus Aufgabe 3 von Übungsblatt 4:
- *kluge*
- *schöne*
- *dicke*

In [24]:
grammar2 = """
'gibt' -> 'Mann' | 'Frau' | 'Buch'
'schenkt' -> 'Mann' | 'Frau' | 'Buch'
'schläft' -> 'Mann' | 'Buch'
'gefällt' -> 'Frau' | 'Buch' 
'kennt' -> 'Frau' | 'Buch' 
'Mann' -> 'der'
'Frau' -> 'der' | 'die'
'Buch' -> 'das'

'Frau' -> 'kluge' | 'schöne' | 'dicke'
'Mann' -> 'kluge' | 'schöne' | 'dicke'
'Buch' -> 'kluge' | 'schöne' | 'dicke'

"""

In [30]:
test_sentences.extend([
    "die kluge schöne Frau kennt das Buch",
    "der schöne kluge Mann gibt der Frau das dicke Buch",
    "das dicke schöne kluge Buch schläft"
])

In [25]:
test_dep_grammar(grammar2,test_sentences)

Satz 1: der Mann gibt der Frau das Buch
     gibt     
 ┌────┼────┐   
Mann Frau Buch
 │    │    │   
der  der  das 

Satz 2: der Mann schläft
schläft
   │    
  Mann 
   │    
  der  

Satz 3: das Buch gefällt der Frau
     gefällt     
 ┌──────┴─────┐   
Buch         Frau
 │            │   
das          der 

Satz 4: die Frau kennt das Buch
     kennt     
 ┌─────┴────┐   
Frau       Buch
 │          │   
die        das 

Satz 5: die kluge schöne Frau kennt das Buch
          kennt            
      ┌─────┴───────────┐   
     Frau              Buch
 ┌────┼───────────┐     │   
die kluge       schöne das 

Satz 6: der schöne kluge Mann gibt der Frau das dicke Buch
            gibt                    
      ┌──────┴────┬────────┐         
     Mann        Frau     Buch      
 ┌────┼──────┐    │    ┌───┴─────┐   
der schöne kluge der  das      dicke

Satz 7: das dicke schöne kluge Buch schläft
    schläft             
       │                 
      Buch              
 ┌─────┼──────┬──

---
## Aufgabe 3: Manuelle Dependenzanalyse

#### Stellen Sie folgenden Satz im Dependenzmodell dar:
- Die neue Kollegin fährt mit der S-Bahn zu ihrem neuen Arbeitsplatz.

#### (a) Ergänzen Sie dafür zunächst den Konstituentenbaum mit Angabe der perkolierten (hochgereichten) Köpfe

(`head(S) = head(VP)`)

(verwendete Regeln für Baum:
`S=NP+VP`, `NP=DET+(ADJ)+N`, `PP=P+NP`, `VP=V+(PP)*`)

In [ ]:
from nltk import Tree

In [33]:
np1 = Tree(
    'NP (Kollegin)',
    [Tree('DET', ['Die']), Tree('ADJ', ['neue']), Tree('N', ['Kollegin'])]
)
vp = Tree(
    'VP (fährt)',
    [
        Tree('V', ['fährt']),
        Tree('PP (mit)', [
            Tree('P', ['mit']),
            Tree('NP (S-Bahn)', [
                Tree('DET', ['der']),
                Tree('N', ['S-Bahn'])
            ])
        ]),
        Tree('PP (zu)', [
            Tree('P', ['zu']),
            Tree('NP (Arbeitsplatz)', [
                Tree('DET', ['ihrem']),
                Tree('ADJ', ['neuen']),
                Tree('N', ['Arb.'])
            ])
        ])
    ]
)
s = Tree(
    'S (fährt)',
    [np1, vp]
)
s.pretty_print(unicodelines=True)

                                     S (fährt)                                                                
          ┌──────────────────────────────┴─────────────────────┐                                               
          │                                                VP (fährt)                                         
          │                  ┌───────────┬─────────────────────┴───────────────────┐                           
          │                  │        PP (mit)                                  PP (zu)                       
          │                  │    ┌──────┴──────────┐                  ┌───────────┴────────────┐              
    NP (Kollegin)            │    │            NP (S-Bahn)             │                NP (Arbeitsplatz)     
 ┌────────┼──────────┐       │    │      ┌──────────┴──────────┐       │    ┌───────────────────┼──────────┐   
DET      ADJ         N       V    P     DET                    N       P   DET                 ADJ         N

#### (b) Fertigen Sie nun eine Tabelle an, in der Sie zu jedem Kopfwort seine Dependenten notieren. Für jede Gruppe unmittelbarer Konstituenten einer Phrase (Ko-Konstituenten) gilt dabei, dass die nicht-hochgereichten Köpfe die Dependenten des hochgereichten Kopfes sind (Nicht-Köpfe in der Phrase sind abhängig vom Kopf der Phrase).



|Kopf|Dependenten|
|----|-----------|
|fährt| Kollegin, mit, zu|
|Kollegin| Die, neue|
|mit| S-Bahn|
|zu| Arbeitsplatz |
|Arbeitsplatz| ihrem, neuen|
|S-Bahn|der|

#### (c) Konvertieren Sie Ihre Tabelle nun in einen Dependenzbaum (Stemma).

In [34]:
dep_tree = Tree(
    "fährt",
    [
        Tree("Kollegin", ["die","neue"]),
        Tree("mit", [Tree("S-Bahn", ['der'])]),
        Tree("zu", [Tree("Arbeitsplatz", ["ihrem", "neuen"])])
    ]
)

dep_tree.pretty_print()

                  fährt                          
        ____________|_______________              
       |           mit              zu           
       |            |               |             
    Kollegin      S-Bahn       Arbeitsplatz      
  _____|______      |       ________|_________    
die          neue  der   ihrem              neuen



#### (d) Versehen Sie den Dependenzbaum mit den Labeln des Universal-Dependency-Tagsets: http://universaldependencies.org/u/dep/ (+ `prep` für Präposition als direkter Dependent zum Verb und `pcomp` als dessen Dependent)

#### Zeichnen Sie den entsprechenden Dependenzbaum (im MALT/CONLL-Format):


In [36]:
sent_nr = """
1 Die 3 det 
2 neue 3 amod
3 Kollegin 4 nsubj
4 fährt 0 ROOT
5 mit 4 PREP
6 der 7 det
7 S-Bahn 5 PCOMP
8 zu 4 PREP
9 ihrem 11 det
10 neuen 11 amod
11 Arbeitsplatz 8 PCOMP
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)
tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

                                     fährt(ROOT)                                      
                ┌─────────────────────────┼────────────────────────┐                   
                │                     mit(PREP)                 zu(PREP)              
                │                         │                        │                   
         Kollegin(nsubj)            S-Bahn(PCOMP)            Arbeitsplatz(            
                │                         │                      PCOMP)               
   ┌────────────┴────────────┐            │           ┌────────────┴────────────┐      
Die(det)                 neue(amod)    der(det)   ihrem(det)               neuen(amod)



#### (e) Fertigen Sie einen weiteren Dependenzgraphen für die alternative Analse der Präposition als Kasus-Marker (`case`) an.

In [38]:
dep_tree = Tree(
    "fährt",
    [
        Tree("Kollegin", ["die","neue"]),
        Tree("S-Bahn", [ 'mit','der']),
        Tree("Arbeitsplatz", ["zu","ihrem", "neuen"])
    ]
)

dep_tree.pretty_print()

                             fährt                       
        _______________________|____________              
    Kollegin          S-Bahn           Arbeitsplatz      
  _____|______      ____|______      _______|_________    
die          neue mit         der   zu    ihrem     neuen



#### Zeichnen Sie den entsprechenden Dependenzbaum (im MALT/CONLL-Format):


In [40]:
sent_nr = """
1 Die 3 det 
2 neue 3 amod
3 Kollegin 4 nsubj
4 fährt 0 ROOT
5 mit 7 CASE
6 der 7 det
7 S-Bahn 4 NMOD/OBL
8 zu 11 CASE
9 ihrem 11 det
10 neuen 11 amod
11 Arbeitsplatz 4 NMOD/OBL
"""


sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)
tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)   

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

                                                               fährt(ROOT)                                   
                ┌────────────────────────────────────┬──────────────┴─────────────────────┐                   
         Kollegin(nsubj)                      S-Bahn(NMOD/OBL)                      Arbeitsplatz(            
                │                                    │                                NMOD/OBL)              
   ┌────────────┴────────────┐          ┌────────────┴──────────────┐         ┌───────────┼────────────┐      
Die(det)                 neue(amod) mit(CASE)                    der(det)  zu(CASE)   ihrem(det)  neuen(amod)



#### (f) Welche Konstituenten lassen sich anhand der Struktur des Dependenzbaumes identifizieren?

- [Die neue Kollegin]
- [mit der S-Bahn]
- [zu ihrem neuen Arbeitsplatz]
- [Die neue Kollegin fährt mit der S-Bahn zu ihrem neuen Arbeitsplatz]

---
# Hausaufgaben 

---
## Aufgabe 4: Manuelle Dependenzanalyse

#### Stellen Sie folgenden Satz im Dependenzmodell dar:
- Ich kenne einen älteren Bruder von zwei sehr lebhaften Mädchen.

#### (a) Erstellen Sie dafür zunächst einen Konstituentenbaum mit Angabe der perkolierten (hochgereichten) Köpfe
(`head(S) = head(VP)`)

(Regeln für Baum:
`S=NP+VP`, `NP=DET+(ADJP)+N+(PP)` oder `NP=PRON` oder `NP=NUM+(ADJP)+N`, `PP=P+NP`, `ADJP=(ADV)+ADJ`, `VP=V+NP`)

Bestimmen Sie die Köpfe dieses Mal sofort gemäß der Primacy of Content Words (http://universaldependencies.org/u/overview/syntax.html); vgl. Aufgabe 3 (e)

In [ ]:
#TEMPLATE:
np1 = Tree(
    'NP (KOPF)',
    [Tree('PRON', ['Ich'])]
)
vp = Tree(
    'VP (KOPF)',
    [Tree('V', ['kenne'])]
)
s = Tree(
    'S',
    [np1, vp]
)

s.pretty_print(unicodelines=True)

#### (b) Fertigen Sie nun eine Tabelle an, in der Sie zu jedem Kopfwort seine Dependenten notieren. Für jede Gruppe unmittelbarer Konstituenten einer Phrase (Ko-Konstituenten) gilt dabei, dass die nicht-hochgereichten Köpfe die Dependenten des hochgereichten Kopfes sind (Nicht-Köpfe in der Phrase sind abhängig vom Kopf der Phrase).


|Kopf|Dependenten|
|----|-----------|



#### (c) Konvertieren Sie Ihre Tabelle nun in einen Dependenzbaum (Stemma).

In [ ]:
dep_tree = Tree(
    "H",
    ["H1", "H2", "..."]
)

dep_tree.pretty_print()

#### (d) Versehen Sie den Dependenzbaum mit den Labeln des Universal-Dependency-Tagsets: http://universaldependencies.org/u/dep/

#### Zeichnen Sie den entsprechenden Dependenzbaum (im MALT/CONLL-Format):

In [ ]:
sent_nr = """
1 H3 3 rel 
2 H2 3 rel
3 H1 0 ROOT 
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)
tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)   

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})